In [47]:
import logging
import os

import fiona
from shapely.geometry import (
    box, mapping, shape, MultiPolygon,
    Polygon
)
import python_gis.poc.util.log_config

logger = logging.getLogger(__name__)

In [3]:
! pwd

/opt/diml-service/notebooks


In [59]:
out_path = os.path.join(os.environ["DATA_DIR"], 'shapefile_out')
shapefile_path = os.path.join(out_path, 'States_US_clean.shp')
nohole_path = os.path.join(out_path, 'States_US_noholes.shp')
grid_path = os.path.join(out_path, 'States_US_shgrid.shp')

## Hole Extraction

In [ ]:
%%time

with fiona.open(shapefile_path) as src:
#     logger.info(f"Driver: {src.driver}, CRS: {src.crs}")
#     logger.info(f"Schema: {src.schema}")

    with fiona.open(new_path, 'w', driver='ESRI Shapefile', 
                    schema=src.schema.copy(), crs=src.crs) as tgt:
        
        for i, rec in enumerate(src):
            polys = []    

            logger.info(f"State: {rec['properties']['State_Name']}")

            try:
                poly = shape(rec['geometry'])
            except Exception as e:
                logger.exception(f"Error reading geometry: {rec['properties']}", e)
                raise

            # Begin function (geometry)
            if poly.geom_type == 'Polygon':

                if len(poly.interiors) > 0:
                    poly = Polygon(poly.exterior)
                
            elif poly.geom_type == 'MultiPolygon':
                for ring in poly.geoms:
                    if len(ring.interiors) > 0:
                        polys.append(Polygon(ring.exterior))                        
                    else:
                        polys.append(ring)

                poly = MultiPolygon(polys)
                
            else:  ## move to assert
                raise Exception(f"Unsupported geometry type: {poly.geom_type}.")
            # End function, return geometry    
            
            rec['geometry'] = mapping(poly)
            tgt.write(rec)

            polys = poly = None
                
print('Completed')

In [ ]:
%%time

with fiona.open(shapefile_path) as src:
#     logger.info(f"Driver: {src.driver}, CRS: {src.crs}")
#     logger.info(f"Schema: {src.schema}")

        
    for i, rec in enumerate(src):

        try:
            poly = shape(rec['geometry'])
        except Exception as e:
            logger.exception(f"Error reading geometry: {rec['properties']}", e)
            raise

        if poly.geom_type == 'MultiPolygon':
            for g in poly.geoms:
                if len(g.interiors) > 0:
                    print(f"State: {rec['properties']['State_Name']}, {len(g.interiors)}")
                    
                    for ring in g.interiors:
                        print(len(ring.coords))
                
print('Completed')